In [ ]:
import numpy as np
import os

import basd
import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client
from dask.distributed import LocalCluster
import multiprocessing as mp
import numpy as np
import xarray as xr

In [ ]:
input_dir = '../basd/data'
output_dir = '../basd/data/output'

#### Reading NetCDF file
Reading in observational precipitation data from 1979-2014 supplied in the data directory.

In [ ]:
pr_obs_hist = xr.open_dataset(os.path.join(input_dir, 'pr_obs-hist_fine_1979-2014.nc'))
pr_sim_hist = xr.open_dataset(os.path.join(input_dir, 'pr_sim-hist_coarse_1979-2014.nc'))
pr_sim_fut = xr.open_dataset(os.path.join(input_dir, 'pr_sim-fut_coarse_2065-2100.nc'))

### Running Bias Adjustment
#### Initializing our first `Adjustment` object
Here we are providing the three required data arrays and a variable name 'pr' representing the daily mean precipitation variable. Precipitation is assumed to follow a gamma distribution, and thus requires a lower bound and lower threshold parameter. We also set trend preservation mode to be 'mixed'. Thus, we set the parameter object directly specifying:

* Lower bound = 0
* Lower threshold = 0.0000011574
* Trend preservation = mixed
* Distribution = gamma

Otherwise, we are keeping the default parameter values for the bias adjustment.

In [ ]:
params = basd.Parameters(lower_bound=0, lower_threshold=0.0000011574,
                         trend_preservation='mixed',
                         distribution='gamma',
                         if_all_invalid_use=0, n_iterations=10)
ba = basd.Adjustment(pr_obs_hist, pr_sim_hist, pr_sim_fut,
                     'pr', params, remap_grid=True )

#### Running Bias Adjustment
Here we run the bias adjustment for just one grid cell. We pass in a tuple with the indexes which we wish to adjust, (0,0), which in this case correspond to the coordinates 53.5 N, 12.5 E.

In [ ]:
loc = dict(lat=1, lon=1)
sim_fut_ba_loc = ba.adjust_bias_one_location(loc)

In [ ]:
sim_fut_ba_loc.plot_hist(scale='log')

In [ ]:
with LocalCluster(processes=True) as cluster, Client(cluster) as client:
    pr_sim_fut_ba = ba.adjust_bias(lat_chunk_size=1, lon_chunk_size=1, path=output_dir)

In [ ]:
cluster = LocalCluster(processes=True)
client = Client(cluster)
client.dashboard_link

In [ ]:
pr_sim_fut_ba = ba.adjust_bias(lat_chunk_size=1, lon_chunk_size=1)
# ba.save_adjustment_nc(output_dir)

In [ ]:
cluster.close()

# Statistical Downscaling
We're now going to start to look at downscaling simulated data to observational data resolution using the statistical downscaling algorithm.

In [ ]:
sd = basd.Downscaler(pr_obs_hist, pr_sim_fut_ba, 'pr', params)

In [ ]:
loc = dict(lat=0, lon=0)
out = sd.downscale_one_location(loc)

In [ ]:
sd.downscale()

# Dask Implementation

In [ ]:
data_path = '../basd/data'

In [ ]:
pr_obs_hist_fine = xr.open_mfdataset(os.path.join(data_path, 'pr_obs-hist_fine*'))
pr_sim_hist = xr.open_mfdataset(os.path.join(data_path, 'pr_sim-hist_coarse*'))
pr_sim_fut = xr.open_mfdataset(os.path.join(data_path, 'pr_sim-fut_coarse*'))

In [ ]:
pr_obs_hist = basd.project_onto(pr_obs_hist_fine, pr_sim_hist, 'pr')

In [ ]:
pr_obs_hist = pr_obs_hist.chunk(dict(lat=1, lon=1, time=-1))
pr_sim_hist = pr_sim_hist.chunk(dict(lat=1, lon=1, time=-1))
pr_sim_fut = pr_sim_fut.chunk(dict(lat=1, lon=1, time=-1))

In [ ]:
pr_obs_hist['pr'] = pr_obs_hist['pr'].transpose('lon', 'lat', 'time')
pr_sim_hist['pr'] = pr_sim_hist['pr'].transpose('lon', 'lat', 'time')
pr_sim_fut['pr'] = pr_sim_fut['pr'].transpose('lon', 'lat', 'time')

In [ ]:
params = basd.Parameters(lower_bound=0, lower_threshold=0.0000011574,
                         trend_preservation='mixed',
                         distribution='gamma',
                         if_all_invalid_use=0, n_iterations=10)

In [ ]:
days, month_numbers, years = basd.time_scraping({'obs_hist': pr_obs_hist['pr'],
                    'sim_hist': pr_sim_hist['pr'],
                    'sim_fut': pr_sim_fut['pr']})